<a href="https://colab.research.google.com/github/Hameeda1512/S5-Assignment-Solutions/blob/main/S5_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms


In [ ]:
# Model paramters: 11730, accuray: >99.4

import torch.nn as nn
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv = nn.Sequential(
            nn.Conv2d(1, 4, 3, padding=1),
            nn.ReLU(),
            # nn.Conv2d(4, 16, 3, padding=1),
            # nn.ReLU(),
            nn.BatchNorm2d(4),
            nn.MaxPool2d(2,2),
            nn.Dropout(0.50),
            nn.Conv2d(4, 16, 3, padding=1),
            nn.ReLU(),
            # nn.Conv2d(16, 32, 3, padding=1),
            # nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.MaxPool2d(2,2),
            nn.Dropout(0.50),
            nn.Conv2d(16, 32, 3),
            nn.ReLU(),
            nn.Conv2d(32, 10, 3),
            nn.AvgPool2d(3, 2),
        )
 
    def forward(self, x):
        x = self.conv(x)
        
        x = x.view(-1, 10)
        x = F.log_softmax(x, dim=1)
        return x


In [ ]:
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 4, 28, 28]              40
              ReLU-2            [-1, 4, 28, 28]               0
       BatchNorm2d-3            [-1, 4, 28, 28]               8
         MaxPool2d-4            [-1, 4, 14, 14]               0
           Dropout-5            [-1, 4, 14, 14]               0
            Conv2d-6           [-1, 16, 14, 14]             592
              ReLU-7           [-1, 16, 14, 14]               0
       BatchNorm2d-8           [-1, 16, 14, 14]              32
         MaxPool2d-9             [-1, 16, 7, 7]               0
          Dropout-10             [-1, 16, 7, 7]               0
           Conv2d-11             [-1, 32, 5, 5]           4,640
             ReLU-12             [-1, 32, 5, 5]               0
           Conv2d-13             [-1, 10, 3, 3]           2,890
        AvgPool2d-14             [-1, 1

In [ ]:
torch.manual_seed(1)
batch_size = 128
 
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        #transforms.RandomRotation(30),
                        transforms.RandomAffine(degrees=20, translate=(0.1,0.1), scale=(0.9, 1.1)),
                        transforms.ColorJitter(brightness=0.2, contrast=0.2),
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307), (0.3081))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)

In [ ]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
 
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')
 
 
 
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
 
    test_loss /= len(test_loader.dataset)
 
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    return test_loss

In [ ]:
import torch.optim as optim
 
model = Net().to(device)
# optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
optimizer= optim.Adam(model.parameters(), lr= 0.01)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=2) # Learning rate scheduler with patience = 2
 
for epoch in range(1, 15):
    train(model, device, train_loader, optimizer, epoch)
    test_loss = test(model, device, test_loader)

    
    print('Epoch-{0} lr: {1}'.format(epoch, optimizer.param_groups[0]['lr']))
    scheduler.step(test_loss)

loss=0.5424759387969971 batch_id=0:   0%|          | 1/469 [00:00<01:13,  6.35it/s]


Test set: Average loss: 0.1810, Accuracy: 9419/10000 (94.19%)

Epoch-1 lr: 0.01


loss=0.39778342843055725 batch_id=0:   0%|          | 1/469 [00:00<01:09,  6.69it/s]


Test set: Average loss: 0.1397, Accuracy: 9558/10000 (95.58%)

Epoch-2 lr: 0.01


loss=0.40838199853897095 batch_id=0:   0%|          | 1/469 [00:00<01:17,  6.06it/s]

...


Test set: Average loss: 0.0767, Accuracy: 9739/10000 (97.39%)

Epoch-12 lr: 0.001


loss=0.3311799466609955 batch_id=0:   0%|          | 1/469 [00:00<01:09,  6.75it/s]


Test set: Average loss: 0.0773, Accuracy: 9734/10000 (97.34%)

Epoch-13 lr: 0.001


loss=0.33482155203819275 batch_id=468: 100%|██████████| 469/469 [01:11<00:00,  6.54it/s]



Test set: Average loss: 0.0770, Accuracy: 9739/10000 (97.39%)

Epoch-14 lr: 0.001
